# Development Notebook for NYISO Powergrid Data.

In [1]:
import sys
import os
sys.path.append(os.path.abspath('../'))

import calendar
import datetime as dt
import numpy as np
import pandas as pd
import re
from twitterinfrastructure.tools import connect_db, create_table, df_to_table, \
    get_regex_files, output, query

import twitterinfrastructure.nyiso as ny
import importlib
importlib.reload(ny)

print(os.getcwd())

/Users/httran/Documents/projects/twitterinfrastructure


## Compare real-time vs integrated real-time actual power load data.

Mean of real-time hourly values does NOT match integrated real-time value for the corresponding hour.

In [34]:
path = 'data/raw/nyiso/palIntegrated/20121030pal.csv'
df = pd.read_csv(path)
df = df.set_index(['Time Stamp', 'Name'])
df = df.sort_index(level=0)
mean_load = df.loc[(slice('10/30/12 0:00', '10/30/12 0:55'), ['CAPITL']),
                   'Load'].mean()
print('Mean load for CAPITL during hour 0 of 10/30/12: ' + str(mean_load))
# df.head()

path_int = 'data/raw/nyiso/palIntegrated/20121030palIntegrated.csv'
df_int = pd.read_csv(path_int)
df_int = df_int.set_index(['Time Stamp', 'Name'])
df_int = df_int.sort_index(level=0)
df_int.head()
# mean_load = df.loc[(['10/30/12 0:00'], ['CAPITL']), 'Load'].mean()
# print('Integrated load for CAPITL during hour 0 of 10/30/12:' + str(mean_load))

Mean load for CAPITL during hour 0 of 10/30/12: 935.4583333333334


<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th></th>
      <th>Time Zone</th>
      <th>PTID</th>
      <th>Integrated Load</th>
    </tr>
    <tr>
      <th>Time Stamp</th>
      <th>Name</th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th rowspan="5" valign="top">10/30/2012 00:00:00</th>
      <th>CAPITL</th>
      <td>EDT</td>
      <td>61757</td>
      <td>929.5</td>
    </tr>
    <tr>
      <th>CENTRL</th>
      <td>EDT</td>
      <td>61754</td>
      <td>1370.2</td>
    </tr>
    <tr>
      <th>DUNWOD</th>
      <td>EDT</td>
      <td>61760</td>
      <td>313.2</td>
    </tr>
    <tr>
      <th>GENESE</th>
      <td>EDT</td>
      <td>61753</td>
      <td>902.5</td>
    </tr>
    <tr>
      <th>HUD VL</th>
      <td>EDT</td>
      <td>61758</td>
      <td>455.4</td>
    </tr>
  </tbody>
</table>
</div>

## Test import.

In [132]:
# test load
# load_type = 'palIntegrated'
# dl_dir = 'data/raw/nyiso/palIntegrated/'
load_type = 'isolf'
dl_dir = 'data/raw/nyiso/isolf/'
df = ny.load_loaddate('20121030', dl_dir=dl_dir, load_type=load_type)
print(df.head())

         Time Stamp  Capitl  Centrl  Dunwod  Genese  Hud Vl  Longil  Mhk Vl  \
0  10/30/2012 00:00     949    1422     492     892     823    1910     607   
1  10/30/2012 01:00     910    1376     458     860     780    1750     582   
2  10/30/2012 02:00     887    1350     440     834     762    1662     565   
3  10/30/2012 03:00     879    1334     432     824     754    1616     561   
4  10/30/2012 04:00     892    1356     431     834     760    1632     572   

   Millwd  N.Y.C.  North  West  NYISO  
0     243    4740    696  1549  14323  
1     228    4448    694  1489  13575  
2     218    4257    686  1457  13118  
3     212    4159    685  1442  12898  
4     214    4149    684  1457  12981  


In [51]:
# test time zones
summer_str = '06/01/2012 05:00:00'
tz = 'America/New_York'
summer_datetime = pd.Timestamp(summer_str, tz=tz)
summer_datetimeUTC = summer_datetime.tz_convert('UTC')
print('NY summer hour: ' + str(summer_datetime.hour))
print('UTC summer hour: ' + str(summer_datetimeUTC.hour))
print('')

winter_str = '12/01/2012 05:00:00'
tz = 'America/New_York'
winter_datetime = pd.Timestamp(winter_str, tz=tz)
winter_datetimeUTC = winter_datetime.tz_convert('UTC')
print('NY winter hour: ' + str(winter_datetime.hour))
print('UTC winter hour: ' + str(winter_datetimeUTC.hour))

NY summer hour: 5
UTC summer hour: 9

NY winter hour: 5
UTC winter hour: 10


In [191]:
# test that isolf nyiso column is the sum of all zones for that hour
load_type = 'isolf'
dl_dir = 'data/raw/nyiso/isolf/'
df = ny.load_loaddate('20121030', dl_dir=dl_dir, load_type=load_type)
df = df.set_index('Time Stamp')
nyiso = df['NYISO']
df = df[['Capitl', 'Centrl', 'Dunwod', 'Genese', 'Hud Vl', 'Longil', 
         'Mhk Vl', 'Millwd', 'N.Y.C.', 'North', 'West']]
df['total'] = df.sum(axis=1)
df['NYISO'] = nyiso
print('total difference between NYISO column and sum of zones: ' + 
      str(sum(df['total'].values - nyiso)))
print(df.head())

total difference between NYISO column and sum of zones: 0
                  Capitl  Centrl  Dunwod  Genese  Hud Vl  Longil  Mhk Vl  \
Time Stamp                                                                 
10/30/2012 00:00     949    1422     492     892     823    1910     607   
10/30/2012 01:00     910    1376     458     860     780    1750     582   
10/30/2012 02:00     887    1350     440     834     762    1662     565   
10/30/2012 03:00     879    1334     432     824     754    1616     561   
10/30/2012 04:00     892    1356     431     834     760    1632     572   

                  Millwd  N.Y.C.  North  West  total  NYISO  
Time Stamp                                                   
10/30/2012 00:00     243    4740    696  1549  14323  14323  
10/30/2012 01:00     228    4448    694  1489  13575  13575  
10/30/2012 02:00     218    4257    686  1457  13118  13118  
10/30/2012 03:00     212    4159    685  1442  12898  12898  
10/30/2012 04:00     214    4149    6

In [280]:
# test isolf clean (incomplete - need to add next day forecast columns)
load_type = 'isolf'
dl_dir = 'data/raw/nyiso/isolf/'
df = ny.load_loaddate('20121030', dl_dir=dl_dir, load_type=load_type)

to_zoneid = False
zones_path = 'tests/nyiso/raw/nyiso-zones.csv'

# clean column names
df = df.rename(columns={'Time Stamp': 'datetime',
                        'Capitl': 'CAPITL',
                        'Centrl': 'CENTRL',
                        'Dunwod': 'DUNWOD',
                        'Genese': 'GENESE',
                        'Hud Vl': 'HUD VL',
                        'Longil': 'LONGIL',
                        'Mhk Vl': 'MHK VL',
                        'Millwd': 'MILLWD',
                        'N.Y.C.': 'N.Y.C.',
                        'North': 'NORTH',
                        'West': 'WEST'})

# clean datetime
df['datetime'] = pd.DatetimeIndex(df['datetime'], tz='America/New_York', 
                                  ambiguous='infer')

# # set index
df = df.set_index('datetime')

# remove columns
df = df.drop('NYISO', axis=1)

# clean zone_id
if to_zoneid:
    zone_col = 'zone_id'
    if zones_path:
        df_zones = pd.read_csv(zones_path)
        zones = dict(zip(df_zones['zone_name'], df_zones['zone_id']))
        df = df.rename(columns=zones)
    else:
        raise ValueError('Must provide zones_path argument if to_zoneid is '
                         'True.')
else:
    zone_col = 'zone_name'

# reshape dataframe
s = df.stack()
s.index.names = ['datetime', zone_col]
df = pd.DataFrame(s.rename('forecast_load'))
# print(df.head())

# move next day forecasts to columns
datetimes = pd.unique(df.index.get_level_values(level=0))
dates = pd.unique([datetime.date() for datetime in datetimes])
# for date in dates:
date = dates[1]
print(pd.Timestamp(date))
print(date + dt.timedelta(hours=23))
df.loc[(slice(pd.Timestamp(date), 
              pd.Timestamp(date) + dt.timedelta(hours=23)), 
        slice(None)), 'forecast_load']

2012-10-31 00:00:00
2012-10-31


datetime                   zone_name
2012-10-31 00:00:00-04:00  CAPITL        936
                           CENTRL       1452
                           DUNWOD        495
                           GENESE        900
                           HUD VL        846
                           LONGIL       1794
                           MHK VL        615
                           MILLWD        247
                           N.Y.C.       4720
                           NORTH         688
                           WEST         1551
2012-10-31 01:00:00-04:00  CAPITL        910
                           CENTRL       1387
                           DUNWOD        459
                           GENESE        858
                           HUD VL        797
                           LONGIL       1694
                           MHK VL        576
                           MILLWD        231
                           N.Y.C.       4417
                           NORTH         692
                  

In [98]:
# test full import (with clean function embedded)
dl_dir = 'data/raw/nyiso/palIntegrated-2012/'
to_zoneid = True
zones_path = 'tests/nyiso/raw/nyiso-zones.csv'
db_path = 'data/processed/nyiso-2012.db'
overwrite = True
verbose = 1

if to_zoneid:
    zone_str = 'zone_id'
else:
    zone_str = 'zone_name'

# get files
pattern = re.compile('\d{8}palIntegrated_csv.zip')
files = get_regex_files(dl_dir, pattern=pattern, verbose=verbose)

# create load table (if needed)
create_sql = """
            CREATE TABLE IF NOT EXISTS load (
                rowid INTEGER PRIMARY KEY,
                datetime TEXT,
                {zone_str} TEXT,
                integrated_load REAL
            ); """.format(zone_str=zone_str)
indexes = ['CREATE INDEX IF NOT EXISTS datetime_{zone_str} '
           'ON load (datetime, {zone_str});'.format(zone_str=zone_str)]
create_table(db_path, 'load', create_sql, indexes=indexes, overwrite=overwrite,
             verbose=verbose)

# load, clean, and import load data into table
import_num = 0
# for file in files:
file = files[10]

if verbose >= 1:
    output('Started importing \"' + file + '\".')

date = pd.Timestamp(file[0:8]).date()
last_day = calendar.monthrange(date.year, date.month)[1]
start_date = pd.Timestamp(year=date.year, month=date.month, day=1)
end_date = pd.Timestamp(year=date.year, month=date.month, day=last_day)
dates = pd.date_range(start_date, end_date)
# for date in dates:
date = dates[3]
date_str = date.strftime('%Y%m%d')
df = ny.load_loaddate(date_str, load_type='palIntegrated',
                      dl_dir=dl_dir, verbose=verbose)

# clean #########################################
# clean column names
df = df.rename(columns={'Time Stamp': 'datetime',
                        'Time Zone': 'timezone',
                        'Name': 'zone_name',
                        'Integrated Load': 'integrated_load'})

# clean datetime
df['datetime'] = pd.to_datetime(df['datetime'], format='%m/%d/%Y %H:%M:%S')
offset = df['timezone'].replace({'EDT': pd.Timedelta('4 hours'), 
                                 'EST': pd.Timedelta('5 hours')})
df['datetimeUTC'] = offset + pd.to_datetime(df['datetime'],
                                            format='%m/%d/%Y %H:%M:%S')
df['datetimeUTC'] = [dtUTC.tz_localize(tz='UTC') for dtUTC in df['datetimeUTC']]

# clean zone_id
if to_zoneid:
    zone_col = 'zone_id'
    if zones_path:
        df_zones = pd.read_csv(zones_path)
        zones = dict(zip(df_zones['zone_name'], df_zones['zone_id']))
        df['zone_id'] = df['zone_name'].replace(zones)
    else:
        raise ValueError('Must provide zones_path argument if to_zoneid is '
                         'True.')
else:
    zone_col = 'zone_name'

# remove columns
df = df[['datetimeUTC', zone_col, 'integrated_load']]

# set index
df = df.set_index(['datetimeUTC', zone_col])
df = df.sort_index(level=0)
# #########################################

import_num += 1

df_write = df.reset_index()
df_write['datetimeUTC'] = df_write['datetimeUTC'].dt.tz_localize(None)
df_to_table(db_path, df_write, table='load', overwrite=False)
del df_write

# df.xs(1, level='zone_id')
# df_write.loc[df_write[zone_col] == 1]

if verbose >= 1:
    output('Finished importing \"' + file + '\".')

output('Finished importing ' + str(import_num) + 
       ' files from \"{dl_dir}\".'.format(dl_dir=dl_dir))

2018-06-25 16:32:55 : 12 matching files in "data/raw/nyiso/palIntegrated-2012/".

2018-06-25 16:32:55 : Dropped load table (if exists).

2018-06-25 16:32:55 : Created new (if not exists) load table.

2018-06-25 16:32:55 : Started importing "20121101palIntegrated_csv.zip".



## Test standardization.

In [177]:
# create expected_load table
db_path = 'data/processed/nyiso-2012.db'
summary_table = 'load'
zones_path = 'data/raw/nyiso/nyiso-zones.csv'
start_ref = pd.Timestamp('2012-01-01 00:00:00', tz='America/New_York')
end_ref = pd.Timestamp('2012-12-31 23:59:59', tz='America/New_York')
datetimeUTC_range_ref = (start_ref.tz_convert(tz='UTC').tz_localize(None), 
                         end_ref.tz_convert(tz='UTC').tz_localize(None))
start = pd.Timestamp('2012-10-22 00:00:00', tz='America/New_York')
end = pd.Timestamp('2012-11-02 23:59:59', tz='America/New_York')
datetimeUTC_range = (start.tz_convert(tz='UTC').tz_localize(None),
                     end.tz_convert(tz='UTC').tz_localize(None))

title = 'sandy'
verbose = 1
datetimeUTC_range_excl = datetimeUTC_range

table = 'expected_load_{title}'.format(title=title)
if verbose >= 1:
    output('Started creating or updating {table} table.'.format(
        table=table))

# query range of zone_id values to consider
df_zones = pd.read_csv(zones_path)
zones = df_zones['zone_id'].unique()
del df_zones

# query reference data
if datetimeUTC_range_excl:
    sql = """
        SELECT datetimeUTC, zone_id, integrated_load
        FROM {summary_table}
        WHERE
            (datetimeUTC BETWEEN "{start_datetime}" AND "{end_datetime}")
            AND (datetimeUTC NOT BETWEEN "{start_datetime_excl}" AND 
                "{end_datetime_excl}")
    ;""".format(summary_table=summary_table,
                start_datetime=datetimeUTC_range_ref[0],
                end_datetime=datetimeUTC_range_ref[1],
                start_datetime_excl=datetimeUTC_range_excl[0],
                end_datetime_excl=datetimeUTC_range_excl[1])
else:
    sql = """
        SELECT datetimeUTC, zone_id, integrated_load
        FROM {summary_table}
        WHERE
            (datetimeUTC BETWEEN "{start_datetime}" AND "{end_datetime}")
    ;""".format(summary_table=summary_table,
                start_datetime=datetimeUTC_range_ref[0],
                end_datetime=datetimeUTC_range_ref[1])
df = query(db_path, sql)

# add dayofweek (0 = Monday) and hour (0-23)
df['datetimeUTC'] = pd.to_datetime(df['datetimeUTC'])
df['datetimeUTC'] = [dtUTC.tz_localize(tz='UTC') for dtUTC in
                     df['datetimeUTC']]
df['datetime'] = [dtUTC.tz_convert(tz='America/New_York') for dtUTC in
                  df['datetimeUTC']]
df['dayofweek'] = df['datetime'].dt.dayofweek
df['hour'] = df['datetime'].dt.hour

# calculate mean and variance for each dayofweek-hour-zone combination
expected = []
dayofweek = 3
hour = 16
zone = 1
for dayofweek in range(7):
    for hour in range(24):
        for zone in zones:
            # filter to current dayofweek, hour, and zone
            df_filter = df[(df['dayofweek'] == dayofweek) &
                           (df['hour'] == hour) &
                           (df['zone_id'] == zone)]

            # calculate mean and variance
            if not df_filter.empty:
                mean_integrated_load = np.mean(
                    df_filter['integrated_load'].values)
                var_integrated_load = np.var(
                    df_filter['integrated_load'].values)
                num_rows = df_filter.shape[0]
                expected.append([dayofweek, hour, zone,
                                 mean_integrated_load, var_integrated_load,
                                 num_rows])
            else:
                expected.append([dayofweek, hour, zone,
                                 np.nan, np.nan, np.nan])
df_exp = pd.DataFrame(expected,
                      columns=['dayofweek', 'hour', 'zone_id',
                               'mean_integrated_load',
                               'var_integrated_load', 'num_rows'])
df_exp.set_index(['dayofweek', 'hour', 'zone_id'])

# # create table
# sql = """
#         CREATE TABLE IF NOT EXISTS {table} (
#             rowid INTEGER PRIMARY KEY,
#             dayofweek INTEGER,
#             hour INTEGER,
#             zone_id INTEGER,
#             mean_integrated_load FLOAT,
#             var_integrated_load FLOAT,
#             num_rows INTEGER
#         ); """.format(table=table)
# create_table(db_path=db_path, table=table, create_sql=sql, indexes=[],
#              overwrite=overwrite, verbose=verbose)
# 
# # write data to table
# df_to_table(db_path, df_exp, table=table, overwrite=False,
#             verbose=verbose)
# 
# if verbose >= 1:
#     output('Finished creating or updating {table} table. Dataframe shape '
#            'is '.format(table=table) + str(df_exp.shape) + '.')

In [247]:
# test create standard_load table
db_path = 'data/processed/nyiso-2012.db'
summary_table = 'load'
expected_table = 'expected_load_sandy'

start = pd.Timestamp('2012-10-22 00:00:00', tz='America/New_York')
end = pd.Timestamp('2012-11-02 23:59:59', tz='America/New_York')
datetimeUTC_range = (start.tz_convert(tz='UTC').tz_localize(None),
                     end.tz_convert(tz='UTC').tz_localize(None))
title = 'sandy'
min_num_rows = 1
verbose = 1

table = 'standard_load_{title}'.format(title=title)
if verbose >= 1:
    output('Started creating or updating {table} table.'.format(
        table=table))

# query expected values calculated from at least min_num_rows data points
sql = """
        SELECT * FROM {expected_table} 
        WHERE num_rows >= {min_num_rows};""".format(
    expected_table=expected_table, min_num_rows=min_num_rows)
df_exp = query(db_path, sql)
df_exp = df_exp[['dayofweek', 'hour', 'zone_id', 'mean_integrated_load',
                 'var_integrated_load']]

# query data to standardize
sql = """
        SELECT datetimeUTC, zone_id, integrated_load
        FROM {summary_table}
        WHERE
            datetimeUTC BETWEEN "{start_datetime}" AND "{end_datetime}";
        """.format(summary_table=summary_table,
                   start_datetime=datetimeUTC_range[0],
                   end_datetime=datetimeUTC_range[1])
df = query(db_path, sql)

# add dayofweek (0 = Monday) and hour (0-23)
df['datetimeUTC'] = pd.to_datetime(df['datetimeUTC'])
df['datetimeUTC'] = [dtUTC.tz_localize(tz='UTC') for dtUTC in
                     df['datetimeUTC']]
df['datetime'] = [dtUTC.tz_convert(tz='America/New_York') for dtUTC in
                  df['datetimeUTC']]
df['dayofweek'] = df['datetime'].dt.dayofweek
df['hour'] = df['datetime'].dt.hour

# calculate z-scores
df = pd.merge(df, df_exp, how='left',
              on=['dayofweek', 'hour', 'zone_id'])
del df_exp
df_std = df[['datetimeUTC', 'zone_id']]
df_std['z_integrated_load'] = \
    (df['integrated_load'] - df['mean_integrated_load']) \
    / df['var_integrated_load']
df_std = df_std.set_index(['datetimeUTC', 'zone_id'])
del df

# create table
sql = """
            CREATE TABLE IF NOT EXISTS {table} (
                rowid INTEGER PRIMARY KEY,
                datetimeUTC TEXT,
                zone_id INTEGER,
                z_integrated_load FLOAT
            ); """.format(table=table)
create_table(db_path=db_path, table=table, create_sql=sql, indexes=[],
             overwrite=overwrite, verbose=verbose)

# write data to table
df_write = df_std.reset_index()
df_write['datetimeUTC'] = df_write['datetimeUTC'].dt.tz_localize(
                None)
df_to_table(db_path, df_write, table=table, overwrite=False, verbose=verbose)
# 
# if verbose >= 1:
#     output('Finished creating or updating {table} table. Dataframe shape '
#            'is '.format(table=table) + str(df_std.shape) + '.')

2018-06-25 20:54:13 : Started creating or updating standard_load_sandy table.

2018-06-25 20:54:13 : Dropped standard_load_sandy table (if exists).



2018-06-25 20:54:13 : Created new (if not exists) standard_load_sandy table.

2018-06-25 20:54:13 : Wrote dataframe to new (if not exists) or existing standard_load_sandy table.



/Users/httran/anaconda3/envs/twitterinfrastructure/lib/python3.6/site-packages/ipykernel_launcher.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


## Test load_forecast import

In [215]:
# test clean

date_str = '20121001'
# date_str = '20121103'
dl_dir = 'data/raw/nyiso/isolf-2012/'
verbose = 2
df = ny.load_loaddate(date_str, load_type='isolf',
                      dl_dir=dl_dir, verbose=verbose)
to_zoneid = True
zones_path = 'data/raw/nyiso/nyiso-zones.csv'

df = ny.clean_isolf(df, to_zoneid=to_zoneid, zones_path=zones_path, verbose=verbose)

# df.head()
# df.to_csv('text.csv')

2018-07-24 16:21:30 : Started loading isolf file for 20121001 from "data/raw/nyiso/isolf-2012/".

2018-07-24 16:21:30 : Finished loading isolf file for 20121001 from "data/raw/nyiso/isolf-2012/".

2018-07-24 16:21:30 : Started cleaning dataframe.

2018-07-24 16:21:30 : Finished cleaning dataframe.



In [10]:
# test write
db_path = 'data/processed/nyiso-2012.db'
zones_path = 'data/raw/nyiso/nyiso-zones.csv'
overwrite = True
verbose = 2

# create_sql = """
#                 CREATE TABLE IF NOT EXISTS load_forecast (
#                     rowid INTEGER PRIMARY KEY,
#                     datetimeNY TEXT,
#                     datetimeUTC TEXT,
#                     zone_id INTEGER,
#                     load_forecast_p0 REAL,
#                     load_forecast_p1 REAL,
#                     load_forecast_p2 REAL,
#                     load_forecast_p3 REAL,
#                     load_forecast_p4 REAL,
#                     load_forecast_p5 REAL,
#                     load_forecast_p6 REAL
#                 ); """
# indexes = ['CREATE INDEX IF NOT EXISTS load_forecast_datetimeNY_zone_id '
#            'ON load_forecast (datetimeNY, zone_id);',
#            'CREATE UNIQUE INDEX IF NOT EXISTS '
#            'load_forecast_datetimeUTC_zone_id ON load_forecast '
#            '(datetimeUTC, zone_id);'
#            ]
# create_table(db_path, 'load_forecast', create_sql, indexes=indexes,
#              overwrite=overwrite,
#              verbose=verbose)

# # clean and write 20121001
# date_str = '20121001'
# dl_dir = 'data/raw/nyiso/isolf-2012/'
# df = ny.load_loaddate(date_str, load_type='isolf',
#                       dl_dir=dl_dir, verbose=verbose)
# df = ny.clean_isolf(df, to_zoneid=True, zones_path=zones_path, verbose=verbose)
# 
# df_write = df.reset_index()
# df_write['datetimeNY'] = df_write['datetimeNY'].dt.tz_localize(None)
# df_write['datetimeUTC'] = df_write['datetimeUTC'].dt.tz_localize(
#     None)
# 
# # connect to database
# conn = connect_db(db_path)
# c = conn.cursor()
# 
# # write each row to table
# for index, row in df_write.iterrows():
#     dtNY = row['datetimeNY']
#     dtUTC = row['datetimeUTC']
#     zone = row['zone_id']
#     val = row.drop(['datetimeNY', 'zone_id', 'datetimeUTC']).dropna()
#     col_name = val.index.values[0] 
#     sql = """
#         INSERT INTO load_forecast (datetimeNY, datetimeUTC, zone_id, 
#             {col_name})
#         VALUES ("{dtNY}", "{dtUTC}", {zone}, {val})
#         ON CONFLICT(datetimeUTC, zone_id) DO
#         UPDATE SET {col_name} = excluded.{col_name}
#     ;""".format(col_name=col_name, val=val[0], dtNY=dtNY, dtUTC=dtUTC,
#                 zone=zone)
#     c.execute(sql)
#     conn.commit()
# conn.close()
# 
# # clean and write 20121002
# date_str = '20121002'
# dl_dir = 'data/raw/nyiso/isolf-2012/'
# df = ny.load_loaddate(date_str, load_type='isolf',
#                       dl_dir=dl_dir, verbose=verbose)
# df = ny.clean_isolf(df, to_zoneid=True, zones_path=zones_path, verbose=verbose)
# 
# df_write = df.reset_index()
# df_write['datetimeNY'] = df_write['datetimeNY'].dt.tz_localize(None)
# df_write['datetimeUTC'] = df_write['datetimeUTC'].dt.tz_localize(
#     None)
# 
# # connect to database
# conn = connect_db(db_path)
# c = conn.cursor()
# 
# # write each row to table
# for index, row in df_write.iterrows():
#     dtNY = row['datetimeNY']
#     dtUTC = row['datetimeUTC']
#     zone = row['zone_id']
#     val = row.drop(['datetimeNY', 'zone_id', 'datetimeUTC']).dropna()
#     col_name = val.index.values[0] 
#     sql = """
#         INSERT INTO load_forecast (datetimeNY, datetimeUTC, zone_id, 
#             {col_name})
#         VALUES ("{dtNY}", "{dtUTC}", {zone}, {val})
#         ON CONFLICT(datetimeUTC, zone_id) DO
#         UPDATE SET {col_name} = excluded.{col_name}
#     ;""".format(col_name=col_name, val=val[0], dtNY=dtNY, dtUTC=dtUTC,
#                 zone=zone)
#     c.execute(sql)
#     conn.commit()
# conn.close()
# 
# # test output
# print(df_write.head())
# db_path = 'data/processed/nyiso-2012.db'
# sql = 'SELECT * FROM load_forecast;'
# df = query(db_path=db_path, sql=sql, parse_dates=False, verbose=2)
# df.to_csv('test2.csv')

2018-07-24 19:41:03 : Started loading isolf file for 20121002 from "data/raw/nyiso/isolf-2012/".

2018-07-24 19:41:03 : Finished loading isolf file for 20121002 from "data/raw/nyiso/isolf-2012/".

2018-07-24 19:41:03 : Started cleaning dataframe.

2018-07-24 19:41:03 : Finished cleaning dataframe.



## Test functions

In [114]:
# test clean_isolf
load_type = 'isolf'
dl_dir = 'tests/nyiso/raw/'
zones_path = 'tests/nyiso/raw/nyiso-zones.csv'
df = ny.load_loaddate('20121030', load_type=load_type, dl_dir=dl_dir)
df = ny.clean_isolf(df, to_zoneid=True, zones_path=zones_path, verbose=0)
print(df.loc[('2012-10-30 01:00:00', 2), ['forecast_load']])
print(df.shape)

assert (df.loc[('2012-10-30 01:00:00', 2), ['forecast_load']].values[0][0]
        == 860) and (df.shape == (1584, 1))


                                   forecast_load
datetime                  zone_id               
2012-10-30 01:00:00-04:00 2                  860
(1584, 1)


In [117]:
# test clean_palint
load_type = 'palIntegrated'
dl_dir = 'tests/nyiso/raw/'
zones_path = 'tests/nyiso/raw/nyiso-zones.csv'
df = ny.load_loaddate('20121030', load_type=load_type, dl_dir=dl_dir)
df = ny.clean_palint(df, to_zoneid=True, zones_path=zones_path, verbose=0)
# print(df[0:30])
print(df.loc[('2012-10-30 06:00:00', 2), ['integrated_load']])
print(df.shape)

assert (df.loc[('2012-10-30 06:00:00', 2), ['integrated_load']].values[0][0]
        == 824.9) and (df.shape == (264, 1))


                                   integrated_load
datetimeUTC               zone_id                 
2012-10-30 06:00:00+00:00 2                  824.9
(264, 1)


In [216]:
# test create_expected_load
db_path = 'tests/nyiso/test.db'
summary_table = 'load'
zones_path = 'tests/nyiso/raw/nyiso-zones.csv'
start_ref = pd.Timestamp('2012-10-01 00:00:00', tz='America/New_York')
end_ref = pd.Timestamp('2012-10-20 23:59:59', tz='America/New_York')
datetimeUTC_range_ref = (start_ref.tz_convert(tz='UTC').tz_localize(None), 
                         end_ref.tz_convert(tz='UTC').tz_localize(None))
start = pd.Timestamp('2012-10-01 00:00:00', tz='America/New_York')
end = pd.Timestamp('2012-10-05 23:59:59', tz='America/New_York')
datetimeUTC_range = (start.tz_convert(tz='UTC').tz_localize(None),
                     end.tz_convert(tz='UTC').tz_localize(None))
_ = ny.create_expected_load(db_path, summary_table, zones_path, 
                            datetimeUTC_range_ref, 
                            datetimeUTC_range_excl=datetimeUTC_range, 
                            title='test', overwrite=True, verbose=0)

sql = 'SELECT * FROM expected_load_test;'
df_test = query(db_path, sql)
print(df_test.head())
print(df_test.loc[2, ['mean_integrated_load']].values[0])

assert (df_test.loc[2, ['mean_integrated_load']].values[0] == 1385) and \
       (df_test.loc[2, ['num_rows']].values[0] == 2) and \
       (df_test.shape == (1848, 7))


   rowid  dayofweek  hour  zone_id  mean_integrated_load  var_integrated_load  \
0      1          0     0        1               1464.80             650.2500   
1      2          0     0        2                843.60             428.4900   
2      3          0     0        3               1385.00             542.8900   
3      4          0     0        4                696.15               6.5025   
4      5          0     0        5                633.75             142.8025   

   num_rows  
0         2  
1         2  
2         2  
3         2  
4         2  
1385.0


,dayofweek,hour,zone_id,mean_integrated_load,var_integrated_load,num_rows
0,0,0,1,1464.80,650.2500,2
1,0,0,2,843.60,428.4900,2
2,0,0,3,1385.00,542.8900,2
3,0,0,4,696.15,6.5025,2
4,0,0,5,633.75,142.8025,2
5,0,0,6,1019.45,93.1225,2
6,0,0,7,800.05,36.6025,2
7,0,0,8,234.60,36.0000,2
8,0,0,9,495.95,7.0225,2
9,0,0,10,4504.55,1718.1025,2


In [246]:
# test create_standard_load
db_path = 'tests/nyiso/test.db'
summary_table = 'load'
expected_table = 'expected_load_test'
start = pd.Timestamp('2012-10-01 00:00:00', tz='America/New_York')
end = pd.Timestamp('2012-10-05 23:59:59', tz='America/New_York')
datetimeUTC_range = (start.tz_convert(tz='UTC').tz_localize(None),
                     end.tz_convert(tz='UTC').tz_localize(None))
_ = ny.create_standard_load(db_path, summary_table, expected_table,
                            datetimeUTC_range, min_num_rows=1, 
                            title='test', overwrite=True, verbose=0)

sql = 'SELECT * FROM standard_load_test;'
df_test = query(db_path, sql)
print(df_test.shape)
print(df_test.loc[2, ['z_integrated_load']].values[0])

assert (round(df_test.loc[2, ['z_integrated_load']].values[0], 2) == -0.04) \
       and (df_test.shape == (1320, 4))


(1320, 4)
-0.03739247361343925


/Users/httran/Documents/projects/twitterinfrastructure/twitterinfrastructure/nyiso.py:420: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  / df['var_integrated_load']


In [120]:
# test import_load
dl_dir = 'tests/nyiso/raw/'
zones_path = 'tests/nyiso/raw/nyiso-zones.csv'
db_path = 'tests/nyiso/test.db'

import_num = ny.import_load(dl_dir, db_path, to_zoneid=True, 
                            zones_path=zones_path, overwrite=True, verbose=0)

sql = 'SELECT * FROM load;'
df_test = query(db_path, sql)
# print(df_test.loc[5])

assert import_num == 31 and \
       df_test.loc[5, 'rowid'] == 6 and \
       df_test.loc[5, 'datetimeUTC'] == '2012-10-01 04:00:00' and \
       df_test.loc[5, 'zone_id'] == 6 and \
       df_test.loc[5, 'integrated_load'] == 1015.9 and \
       df_test.shape == (8184, 4)


2018-06-25 17:03:17 : Finished importing 31 files from "tests/nyiso/raw/".



In [124]:
# test load_loaddate
load_type = 'palIntegrated'
dl_dir = 'tests/nyiso/raw/'
df = ny.load_loaddate('20121030', load_type=load_type, dl_dir=dl_dir)
print(df.loc[10, 'Time Stamp'])
print(df.loc[10, 'Name'])
print(df.loc[10, 'Integrated Load'])

assert (df.loc[10, 'Time Stamp'] == '10/30/2012 00:00:00') and \
       (df.loc[10, 'Name'] == 'WEST') and \
       (df.loc[10, 'Integrated Load'] == 1586.2) and df.shape == (264, 5)

load_type = 'isolf'
dl_dir = 'data/raw/nyiso/isolf/'
df = ny.load_loaddate('20121030', load_type=load_type, dl_dir=dl_dir)
print(df.head())
print(df.loc[4, 'Time Stamp'])
print(df.loc[4, 'N.Y.C.'])
print(df.shape)

assert (df.loc[4, 'Time Stamp'] == '10/30/2012 04:00') and \
       (df.loc[4, 'N.Y.C.'] == 4149) and df.shape == (144, 13)

10/30/2012 00:00:00
WEST
1586.2
         Time Stamp  Capitl  Centrl  Dunwod  Genese  Hud Vl  Longil  Mhk Vl  \
0  10/30/2012 00:00     949    1422     492     892     823    1910     607   
1  10/30/2012 01:00     910    1376     458     860     780    1750     582   
2  10/30/2012 02:00     887    1350     440     834     762    1662     565   
3  10/30/2012 03:00     879    1334     432     824     754    1616     561   
4  10/30/2012 04:00     892    1356     431     834     760    1632     572   

   Millwd  N.Y.C.  North  West  NYISO  
0     243    4740    696  1549  14323  
1     228    4448    694  1489  13575  
2     218    4257    686  1457  13118  
3     212    4159    685  1442  12898  
4     214    4149    684  1457  12981  
10/30/2012 04:00
4149
(144, 13)
